In [2]:
import numpy as np
import math 
import gmpy2

In [3]:
def factors(n): #step 1: prime factorization
    primfact = []
    if n < 0 : 
        primfact.append(-1)
        n = -1 * n
    d = 2
    while d*d <= n:
        while (n % d) == 0:
            primfact.append(d)  
            n //= d
        d += 1
    if n > 1:
        primfact.append(n)
    return primfact

def gen_base(g, alpha, n, p): #step 2: making base from p_i & l_i
    r = []
    for p_i in p:
        r_i = [int(gmpy2.powmod(alpha,int((n*j)/p_i),g)) for j in range(p_i)]
        r.append(r_i)    
    return r

def find_x_i(p, n, g, r, l, alpha, beta): #step 3: findinx x_i for each p_i
    x = []
    for i in range(len(p)):
        x_i = []
        b = int(gmpy2.powmod(beta, int(n/p[i]), g))
        x_0 = r[i].index(b)
        x_i.append(x_0)
        for j in range(1, l[i]):
            power = 0
            for k in range(len(x_i)):
                power -= x_i[k]*(p[i]**k)
            a = int(gmpy2.powmod(alpha, power, g))
            b = int(gmpy2.powmod(beta*a, int(n/(p[i]**(j+1))), g))
            x_new = r[i].index(b)
            x_i.append(x_new)
        x.append(x_i)
    return x

def find_x_val(x_i_s, p, l):#step3: making a system by finding x = x_0 + x_1*p_i + x_2*p_i^2 + ...(mod p_i ^ l_i)
    x_vals = []
    x_mods = []
    for i in range(len(p)):
        val = 0
        for j in range(l[i]):
            val += x_i_s[i][j] * (p[i] ** j)
        x_vals.append(val)
        x_mods. append(p[i]**l[i])
    x_system = [x_vals, x_mods]
    return x_system

def solve_system(x_system, n): #step 4: solve system of linear congruences
    x_vals, m = x_system
    M = np.prod(m)
    M_i = [int(M/m_i) for m_i in m]
    N_i = [int(gmpy2.invert(M_i[i], m[i])) for i in range(len(m))]
    x = 0
    for j in range(len(M_i)):
        x += N_i[j]*M_i[j]*x_vals[j]
    x_0 = x % n
    return x_0

def check_answer(x, alpha, beta, g):
    b = int(gmpy2.powmod(alpha,x,g))
    if b == beta:
        print("right")
    else: 
        print("wrong")

In [4]:
def silver_pohlig_hellman(g, n, alpha, beta):
    fact = factors(n)
    p = list(set(fact))
    l = [fact.count(p_i) for p_i in p]
    print("p=", p)
    print("l=", l)
    r = gen_base(g, alpha, n, p)
    print("r=", r)
    x_i_s = find_x_i(p, n, g, r, l, alpha, beta)
    print("x_i_s=", x_i_s)
    x_system = find_x_val(x_i_s, p, l) #our system to solve: x = value mod p_i ^ l_i
    print("system: [values, mods] =", x_system) 
    x_0 = solve_system(x_system, n)
    return x_0

In [5]:
alpha, beta, g, n = 9704, 13896, 17389, 1242  # тут взяли (n = ord(alpha))
# alpha, beta, g, n = 6, 13, 229, 228
# alpha, beta, g, n = 3, 148, 181, 180
# alpha, beta, g, n = 3, 15, 43, 42 
# alpha, beta, g, n = 5, 11, 73, 72 

x = silver_pohlig_hellman(g, n, alpha, beta)
print("----ANSWER----")
print("x =", x)
check_answer(x, alpha, beta, g)

p= [2, 3, 23]
l= [1, 3, 1]
r= [[1, 17388], [1, 13746, 3642], [1, 11832, 14774, 11740, 4348, 8874, 2386, 8805, 3261, 15350, 10484, 10951, 6793, 2818, 7863, 3866, 9442, 10808, 1550, 11594, 15776, 8106, 9857]]
x_i_s= [[1], [1, 2, 2], [9]]
system: [values, mods] = [[1, 25, 9], [2, 27, 23]]
----ANSWER----
x = 1159
right
